In [20]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///tmp/build/80754af9/scikit-learn_1642617106979/work
scikit-learn-intelex==2021.20220215.212715


In [21]:
!python -V

Python 3.9.12


In [22]:
import pickle
import pandas as pd
import sklearn

In [23]:
sklearn.__version__

'1.0.2'

In [5]:
# !mkdir output

In [24]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [25]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID']
    df = pd.read_parquet(filename, columns=cols)
    
    df['duration'] = (
        df.tpep_dropoff_datetime
          .sub(df.tpep_pickup_datetime)
          .dt
          .total_seconds()
          .div(60)
    )

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [31]:
year = 2023
month = 5

df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')
output_file = f'output/yellow_tripdata_{year:04d}-{month:02d}.parquet'

In [32]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,duration
0,2023-05-01 00:33:13,2023-05-01 00:53:01,138,43,19.800000
1,2023-05-01 00:42:49,2023-05-01 01:11:18,138,262,28.483333
2,2023-05-01 00:56:34,2023-05-01 01:13:39,138,141,17.083333
3,2023-05-01 00:00:52,2023-05-01 00:20:12,138,140,19.333333
4,2023-05-01 00:05:50,2023-05-01 00:19:41,138,263,13.850000


In [33]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [34]:
y_pred

array([23.14355121, 23.53976428, 22.15858502, ..., 11.00516314,
       25.96324411, 13.74985567])

In [35]:
y_pred.mean()

14.242595513316317

# Q1. Standard deviation

In [12]:
y_pred.std()

6.247488852238703

# Q2. Preparing the output

In [13]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [14]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,duration,ride_id
0,2023-03-01 00:06:43,2023-03-01 00:16:43,238,42,10.000000,2023/03_0
1,2023-03-01 00:08:25,2023-03-01 00:39:30,138,231,31.083333,2023/03_1
2,2023-03-01 00:15:04,2023-03-01 00:29:26,140,186,14.366667,2023/03_2
3,2023-03-01 00:49:37,2023-03-01 01:01:05,140,43,11.466667,2023/03_3
4,2023-03-01 00:08:04,2023-03-01 00:11:06,79,137,3.033333,2023/03_4


In [15]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [18]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [19]:
!ls -lh output

total 66M
-rw-rw-rw- 1 codespace codespace 66M Jun 15 22:44 yellow_tripdata_2023-03.parquet
